In [36]:
import mysql.connector as connection
import pandas as pd
from google.cloud import bigquery
import mysql

In [37]:
from dotenv import load_dotenv
import os 

load_dotenv()
myproj      = os.getenv("proj")
myhost      = os.getenv("host")
myuser      = os.getenv("user")
mypasswd    = os.getenv("passwd")
mydatabase  = os.getenv("database")
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "consultas-sql-375300-097b59c460bf.json"

In [38]:
def mysqltobq(table):
    proj=myproj
    result=""
    
    mydb = mysql.connector.connect(
      host=myhost,
      user=myuser,
      passwd=mypasswd,
      database=mydatabase
    )

    allTbls = ["productlines", "products", "offices", "orders", "payments", "customers", "employees", "orderdetails"]

    if (table != "ALL"):        
        result = f"{table} table loaded!"
        query = f"SELECT * FROM {mydatabase}.{table}"
        result_df = pd.read_sql(query,mydb)        
        
        client = bigquery.Client()    
        table_id = proj+f".datos_prueba.{table}"
        table = client.get_table(table_id)
        # mydb.close()
    
        job_config = bigquery.LoadJobConfig(
            schema=table.schema,
            write_disposition="WRITE_TRUNCATE"
        )
        
        job= client.load_table_from_dataframe(
            result_df,
            table_id,
            job_config=job_config
        )

    elif (table == "ALL"):        
        result = "Tables: "
        for t in allTbls:
            
            if ((allTbls[len(allTbls) - 2])==t):
                result = f"{result} {t} &" 
            elif ((allTbls[len(allTbls) - 1])==t):
                result = f"{result} {t} "    
            else:
                result = f"{result} {t}, "

            query = f"SELECT * FROM {mydatabase}.{t}"
            # print(query)
            result_df = pd.read_sql(query,mydb)        
            
            client = bigquery.Client()    
            table_id = proj+f".datos_prueba.{t}"
            table = client.get_table(table_id)            
        
            job_config = bigquery.LoadJobConfig(
                schema=table.schema,
                write_disposition="WRITE_TRUNCATE"
            )
            
            job= client.load_table_from_dataframe(
                result_df,
                table_id,
                job_config=job_config
            )

        result = result+" loaded!"
    mydb.close()
    return result

In [39]:
mysqltobq("ALL")

C:\Users\grego\AppData\Local\Temp\ipykernel_4944\3281181556.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(query,mydb)
C:\Users\grego\AppData\Local\Temp\ipykernel_4944\3281181556.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(query,mydb)
C:\Users\grego\AppData\Local\Temp\ipykernel_4944\3281181556.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(query,mydb)
C:\Users\grego\AppData\Local\Temp\ipykernel_4944\3281181556.py:48: UserWarning: pa

'Tables:  productlines,  products,  offices,  orders,  payments,  customers,  employees & orderdetails  loaded!'